<div style="background-color:green; color:white; padding:10px; font-size:20px">
1 - Setup

In [136]:
!pip install boto3


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [137]:
import boto3
import pandas as pd
import random

# Project Scripts
import data_handling

In [138]:
# Re-freshing the module import for development purposes
from importlib import reload
reload(data_handling)

<module 'data_handling' from '/root/behavior_project/data_handling.py'>

#### Sagemaker User & IAM Credentials

In [139]:
# Create credentials session (log-in to AWS account)
sess = boto3.Session()
# Get information about current User
sts = sess.client("sts")
response = sts.get_caller_identity()

print("Your AWS Account:  717145514721")
print("Logged-in Account:", response["Account"])
print("User ARN:", response["Arn"])

Your AWS Account:  717145514721
Logged-in Account: 717145514721
User ARN: arn:aws:sts::717145514721:assumed-role/AWS-SageMakerFullAccess-Role/SageMaker


### S3 Connection

In [140]:
import boto3

# Create a session using your AWS credentials
s3 = boto3.client("s3")

# List all bucket names
response = s3.list_buckets()

# Get a list of all bucket names from the response
buckets = [bucket["Name"] for bucket in response["Buckets"]]

# Print out the bucket list
print("Bucket List: %s" % buckets)

Bucket List: ['behavior-project', 'sagemaker-us-east-1-717145514721']


<div style="background-color:green; color:white; padding:10px; font-size:20px">
2 - Data Import

#### Inspect Single File

<div style="background-color:lightgreen; color:black; padding:10px; font-size:20px">
2.1 - mPFC Import

In [141]:
from data_handling import load_to_dict

In [142]:
mPFC_raw_path = "s3://behavior-project/raw-data/mPFC-ICSS/"

dtype_conversion_dict = {"float64": "float32", "int64": "bool"}

mPFC_df_dict = load_to_dict(
    s3_url=mPFC_raw_path,
    dtype_dict=dtype_conversion_dict)

In [143]:
# View Loaded Dataframe
key, value = next(iter(mPFC_df_dict.items()))
print(f"Dataframe: {key}")
value.head(3)

Dataframe: C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON


,Time,Centre position X,Centre position Y,In Triangle,In Dot,In Horizontal,In Vertical,Triangle active,Dot active,Horizontal active,Vertical active
0,0.000,NaN,NaN,False,False,True,False,False,False,False,False
1,1.278,475.0,324.0,False,False,True,False,False,False,False,False
2,1.285,475.0,324.0,False,False,True,False,False,False,True,False


In [144]:
from data_handling import create_stim_session_list

In [145]:
# Get List of Animal ID's
animal_ID_list = []
for key, value in mPFC_df_dict.items():
  animal_ID_list.append(key)

# Convert List to DataFrame
mPFC_session_info_df = pd.DataFrame(animal_ID_list, columns = ['session_id'])

Col_Name_List = ['animal_id_1','animal_id_2','virus','stim_target','stim_freq','session_num','pod_num','stim_zone','sens_treat_1','sens_treat_2']

# Parse ID Elements to Separate Columns
mPFC_session_info_df[Col_Name_List] = mPFC_session_info_df['session_id'].str.split('_', expand = True)

# Function that creates stim zone list for all of the animals sessions
mPFC_session_info_df = create_stim_session_list(mPFC_session_info_df)

mPFC_session_info_df.head(3)

,session_id,animal_id_1,animal_id_2,virus,stim_target,stim_freq,session_num,pod_num,stim_zone,sens_treat_1,sens_treat_2,stim_session_list
0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,C58,R10,ChR2,mPFC,30Hz,S01,P02,N,NON,NON,NVN
1,C58_R10_ChR2_mPFC_30Hz_S02_P02_V_NON_NON,C58,R10,ChR2,mPFC,30Hz,S02,P02,V,NON,NON,NVN
2,C58_R10_ChR2_mPFC_30Hz_S03_P02_N_NON_NON,C58,R10,ChR2,mPFC,30Hz,S03,P02,N,NON,NON,NVN


<div style="background-color:lightgreen; color:white; padding:10px; font-size:20px">
2.2 - FR Import

In [146]:
FR_raw_path = "s3://behavior-project/raw-data/FR-ICSS/"

dtype_conversion_dict = {"float64": "float32", "int64": "bool"}

FR_df_dict = load_to_dict(s3_url=FR_raw_path, dtype_dict=dtype_conversion_dict)

In [147]:
# View Loaded Dataframe
key, value = next(iter(FR_df_dict.items()))
print(f"Dataframe: {key}")
value.head(3)

Dataframe: C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON


,Time,Centre position X,Centre position Y,In 1t,In 1d,In 1h,In 1v,In 2t,In 2d,In 2h,...,6hs active,6vs active,7ts active,7ds active,7hs active,7vs active,8ts active,8ds active,8hs active,8vs active
0,0.000,NaN,NaN,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.287,271.0,260.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0.407,261.0,267.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [148]:
# Get List of Animal ID's
animal_ID_list = []
for key, value in FR_df_dict.items():
  animal_ID_list.append(key)

# Convert List to DataFrame
FR_session_info_df = pd.DataFrame(animal_ID_list, columns = ['session_id'])

Col_Name_List = ['animal_id_1','animal_id_2','virus','stim_target','stim_freq','session_num','pod_num','stim_zone','sens_treat_1','sens_treat_2']

# Parse ID Elements to Separate Columns
FR_session_info_df[Col_Name_List] = FR_session_info_df['session_id'].str.split('_', expand = True)

# Function that creates stim zone list for all of the animals sessions
FR_session_info_df = create_stim_session_list(FR_session_info_df)

FR_session_info_df.head(3)

,session_id,animal_id_1,animal_id_2,virus,stim_target,stim_freq,session_num,pod_num,stim_zone,sens_treat_1,sens_treat_2,stim_session_list
0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,C1000,C42A,ChR2,xPVT,F00,S01,P05,N,NON,NON,NTD
1,C1000_C42A_ChR2_xPVT_F20_S02_P05_T_NON_NON,C1000,C42A,ChR2,xPVT,F20,S02,P05,T,NON,NON,NTD
2,C1000_C42A_ChR2_xPVT_F20_S03_P05_D_NON_NON,C1000,C42A,ChR2,xPVT,F20,S03,P05,D,NON,NON,NTD


In [149]:
# pd.set_option('display.max_rows', None)
# FR_df_dict['C604_C31A_ChR2_xBLA_F20_S02_P03_V_NON_NON'].nunique()

In [150]:
# pd.set_option('display.max_rows', None)
# FR_df_dict['C604_C31A_ChR2_xBLA_F00_S01_P03_N_NON_NON'].nunique()

In [151]:
# pd.set_option('display.max_rows', None)
# FR_df_dict['C604_C31A_ChR2_xBLA_F20_S03_P03_T_NON_NON'].nunique()

<div style="background-color:green; color:white; padding:10px; font-size:20px">
3 - Data Cleaning

In [152]:
from data_handling import remove_columns
from data_handling import view_rand_df

In [153]:
def check_same_columns(df_dictionary):
    # Initialize a list to hold the column names of each dataframe
    cols_list = []

    # Iterate over each dataframe in the dictionary
    for key, df in df_dictionary.items():
        cols_list.append(df.columns.tolist())

    # Check if all column lists in cols_list are the same
    return all(cols == cols_list[0] for cols in cols_list)

<div style="background-color:lightgreen; color:black; padding:10px; font-size:20px">
mPFC

In [154]:
col_removal_list = 'Not in 1t','Not in 1d','Not in 1h','Not in 1v'

mPFC_df_dict, removal_summary = remove_columns(mPFC_df_dict, col_removal_list)
print(removal_summary.keys())

dict_keys([])


In [155]:
# Check for Column Uniformity
same_columns = check_same_columns(mPFC_df_dict)
same_columns

True

In [156]:
def rename_columns(rename_col_dict, df_dict):
    # Convert the rename dictionary keys and values to lower case
    rename_col_dict = {k.lower(): v.lower() for k, v in rename_col_dict.items()}

    for key, df in df_dict.items():
        # Convert the DataFrame column names to lower case
        df.columns = df.columns.str.lower()

        # Rename the columns
        df.rename(columns=rename_col_dict, inplace=True)

    return df_dict

In [157]:
rename_col_dict = {'in triangle':'in_t',
                  'in dot':'in_d',
                  'in horizontal':'in_h',
                  'in vertical':'in_v',}

mPFC_df_dict = rename_columns(rename_col_dict, mPFC_df_dict)

In [158]:
# View Results
view_rand_df(df_dict = mPFC_df_dict, info_df = mPFC_session_info_df)

Dataframe: C58_R20_ChR2_mPFC_30Hz_S02_P04_V_NON_NON
stim_session_list: NV-


,time,centre position x,centre position y,in_t,in_d,in_h,in_v,triangle active,dot active,horizontal active,vertical active
0,0.000,NaN,NaN,False,True,False,False,False,False,False,False
1,0.399,475.0,185.0,False,True,False,False,False,False,False,False
2,0.407,475.0,185.0,False,True,False,False,False,True,False,False


#### Remove Irrelevant 'Active Zone' Cols

In [159]:
DF_DICTIONARY = mPFC_df_dict
INFO_DF = mPFC_session_info_df

for key, value in DF_DICTIONARY.items():

  # Grab session activation zone
  active_zone = key.split('_')[7]

  # Grab sessions list for animal
  session_list_str = INFO_DF[INFO_DF['session_id'] == key]['stim_session_list'].to_string(index=False)
  S01 = session_list_str[0] # 1st str element
  S02 = session_list_str[1] # 2nd str element
  S03 = session_list_str[2] # 3rd str element

  if key.split('_')[7] == 'N':
    active_zone = S02 # Grabbing the zone from animal's acquisition session

  column_to_duplicate = None

  # Check which column to duplicate based on active_zone
  if active_zone == 'T':
    column_to_duplicate = next((col for col in value.columns if col.startswith('in') and 't' in col), None)

  elif active_zone == 'D':
    column_to_duplicate = next((col for col in value.columns if col.startswith('in') and 'd' in col), None)

  elif active_zone == 'H':
    column_to_duplicate = next((col for col in value.columns if col.startswith('in') and 'h' in col), None)

  elif active_zone == 'V':
    column_to_duplicate = next((col for col in value.columns if col.startswith('in') and 'v' in col), None)

  # Assert that column_to_duplicate has been set to something other than None
  assert column_to_duplicate is not None, f"No column to duplicate found for active_zone {active_zone} in dataframe {key}"

  # If a column to duplicate was found, duplicate it
  if column_to_duplicate is not None:
    new_column_name = f"session_zone_({active_zone})"
    value[new_column_name] = value[column_to_duplicate]

  # Assert that the new column was created
  assert new_column_name in value.columns, f"New column {new_column_name} not created in dataframe {key}"

  # Drop all columns that contain 'active' in their name
  value.drop([col for col in value.columns if 'active' in col], axis=1, inplace=True)

  # Assert that the new column wasn't dropped
  assert new_column_name in value.columns, f"New column {new_column_name} was dropped from dataframe {key}"

In [160]:
# View Results
view_rand_df(df_dict = mPFC_df_dict, info_df = mPFC_session_info_df)

Dataframe: C59_R30_ChR2_mPFC_30Hz_S02_P06_V_NON_NON
stim_session_list: NVN


,time,centre position x,centre position y,in_t,in_d,in_h,in_v,session_zone_(V)
0,0.000,NaN,NaN,False,False,True,False,False
1,0.859,415.0,391.0,False,False,True,False,False
2,0.868,415.0,391.0,False,False,True,False,False


#### Str Formatting

In [161]:
from data_handling import format_cols

In [162]:
col_rename_dict = {"centre position x":"x_position",
                   "centre position y":"y_position",
                   "in triangle":"in_t",
                   "in dot":"in_d",
                   "in horizontal":"in_h",
                   "in vertical":"in_v"}

mPFC_df_dict = format_cols(mPFC_df_dict, col_rename_dict)

In [163]:
# View Results
view_rand_df(df_dict = mPFC_df_dict, info_df = mPFC_session_info_df)

Dataframe: C58_R12_ChR2_mPFC_30Hz_S02_P04_D_NON_NON
stim_session_list: NDN


,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,session_zone_(d)
0,0.000,NaN,NaN,False,False,True,False,False
1,0.573,480.0,422.0,False,False,True,False,False
2,0.581,480.0,422.0,False,False,True,False,False


In [164]:
next(iter(mPFC_df_dict.values())).columns

Index(['time', 'centre_position_x', 'centre_position_y', 'in_t', 'in_d',
       'in_h', 'in_v', 'session_zone_(v)'],
      dtype='object')

<div style="background-color:lightgreen; color:black; padding:10px; font-size:20px">
FR_ICSS

In [165]:
col_removal_list = 'Not in 1t','Not in 1d','Not in 1h','Not in 1v'

FR_df_dict, removal_summary = remove_columns(FR_df_dict, col_removal_list)
print(removal_summary.keys())

dict_keys(['C907_C39A_ChR2_xPVT_F00_S01_P04_N_NON_NON', 'C907_C39A_ChR2_xPVT_F30_S02_P04_D_NON_NON', 'C907_C39A_ChR2_xPVT_F30_S03_P04_H_NON_NON', 'C908_C39A_ChR2_xPVT_F00_S01_P05_N_NON_NON', 'C908_C39A_ChR2_xPVT_F20_S02_P05_D_NON_NON', 'C908_C39A_ChR2_xPVT_F20_S03_P05_V_NON_NON'])


In [166]:
# Check for Column Uniformity
same_columns = check_same_columns(FR_df_dict)
same_columns

True

### Removing Irrelevant Columns
The FR_ICSS dataset came with zone columns for all pods. Even though one session pertians to only one pod at a time. The code below removes any pod information not relevant to the current session pod.

In [167]:
# Dictionary of 'Keep Column' lists
column_dict = {
    'P01': ['Time', 'Centre position X', 'Centre position Y', 'In 1t', 'In 1d', 'In 1h', 'In 1v', '1ts active', '1ds active', '1hs active', '1vs active'],
    'P02': ['Time', 'Centre position X', 'Centre position Y', 'In 2t', 'In 2d', 'In 2h', 'In 2v', '2ts active', '2ds active', '2hs active', '2vs active'],
    'P03': ['Time', 'Centre position X', 'Centre position Y', 'In 3t', 'In 3d', 'In 3h', 'In 3v', '3ts active', '3ds active', '3hs active', '3vs active'],
    'P04': ['Time', 'Centre position X', 'Centre position Y', 'In 4t', 'In 4d', 'In 4h', 'In 4v', '4ts active', '4ds active', '4hs active', '4vs active'],
    'P05': ['Time', 'Centre position X', 'Centre position Y', 'In 5t', 'In 5d', 'In 5h', 'In 5v', '5ts active', '5ds active', '5hs active', '5vs active'],
    'P06': ['Time', 'Centre position X', 'Centre position Y', 'In 6t', 'In 6d', 'In 6h', 'In 6v', '6ts active', '6ds active', '6hs active', '6vs active'],
    'P07': ['Time', 'Centre position X', 'Centre position Y', 'In 7t', 'In 7d', 'In 7h', 'In 7v', '7ts active', '7ds active', '7hs active', '7vs active'],
    'P08': ['Time', 'Centre position X', 'Centre position Y', 'In 8t', 'In 8d', 'In 8h', 'In 8v', '8ts active', '8ds active', '8hs active', '8vs active'],
}

for key, value in FR_df_dict.items():
    identifier = key.split('_')[6]
    
    if identifier in column_dict:
        # Subset Columns
        FR_df_dict[key] = value[column_dict[identifier]]
    else:
        print('Identifier not found')

In [168]:
# View Dataframe
key, value = next(iter(FR_df_dict.items()))
print(f"Dataframe: {key}")
value.head(3)

Dataframe: C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON


,Time,Centre position X,Centre position Y,In 5t,In 5d,In 5h,In 5v,5ts active,5ds active,5hs active,5vs active
0,0.000,NaN,NaN,False,False,False,False,False,False,False,False
1,0.287,271.0,260.0,False,False,False,False,False,False,False,False
2,0.407,261.0,267.0,False,False,False,False,False,False,False,False


#### Remove Irrelevant Cols & Add 'active_zone'

In [169]:
DF_DICTIONARY = FR_df_dict
INFO_DF = FR_session_info_df

for key, value in DF_DICTIONARY.items():

  # Grab session activation zone
  active_zone = key.split('_')[7]

  # Grab sessions list for animal
  session_list_str = INFO_DF[INFO_DF['session_id'] == key]['stim_session_list'].to_string(index=False)
  S01 = session_list_str[0] # 1st str element
  S02 = session_list_str[1] # 2nd str element
  S03 = session_list_str[2] # 3rd str element

  if key.split('_')[7] == 'N':
    active_zone = S02 # Grabbing the zone from animal's acquisition session

  column_to_duplicate = None
    
  # Check which column to duplicate based on active_zone
  if active_zone == 'T':
    column_to_duplicate = next((col for col in value.columns if col.startswith('In') and 't' in col), None)

  elif active_zone == 'D':
    column_to_duplicate = next((col for col in value.columns if col.startswith('In') and 'd' in col), None)

  elif active_zone == 'H':
    column_to_duplicate = next((col for col in value.columns if col.startswith('In') and 'h' in col), None)

  elif active_zone == 'V':
    column_to_duplicate = next((col for col in value.columns if col.startswith('In') and 'v' in col), None)

  # Assert that column_to_duplicate has been set to something other than None
  assert column_to_duplicate is not None, f"No column to duplicate found for active_zone {active_zone} in dataframe {key}"

  # If a column to duplicate was found, duplicate it
  if column_to_duplicate is not None:
    new_column_name = f"session_zone_({active_zone})"
    value[new_column_name] = value[column_to_duplicate]

  # Assert that the new column was created
  assert new_column_name in value.columns, f"New column {new_column_name} not created in dataframe {key}"

  # Drop all columns that contain 'active' in their name
  value.drop([col for col in value.columns if 'active' in col], axis=1, inplace=True)

  # Assert that the new column wasn't dropped
  assert new_column_name in value.columns, f"New column {new_column_name} was dropped from dataframe {key}"

In [170]:
next(iter(FR_df_dict.values())).columns

Index(['Time', 'Centre position X', 'Centre position Y', 'In 5t', 'In 5d',
       'In 5h', 'In 5v', 'session_zone_(T)'],
      dtype='object')

In [171]:
# View Results
view_rand_df(df_dict = FR_df_dict, info_df = FR_session_info_df)

Dataframe: C485_C25B_ChR2_xILC_F00_S01_P05_N_NON_NON
stim_session_list: NTV


,Time,Centre position X,Centre position Y,In 5t,In 5d,In 5h,In 5v,session_zone_(T)
0,0.000,NaN,NaN,False,False,False,False,False
1,0.284,135.0,208.0,False,False,False,False,False
2,0.395,137.0,207.0,False,False,False,False,False


#### Str Formatting

In [172]:
from data_handling import remove_numbers_from_columns

In [173]:
FR_df_dict = remove_numbers_from_columns(FR_df_dict)
FR_df_dict = format_cols(FR_df_dict, col_rename_dict)

In [174]:
# View Results
view_rand_df(df_dict = FR_df_dict, info_df = FR_session_info_df)

Dataframe: C796_C35B_ChR2_xBLA_F00_S01_P07_N_NON_NON
stim_session_list: NTH


,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,session_zone_(t)
0,0.000,NaN,NaN,False,False,True,False,False
1,0.281,497.0,395.0,False,False,True,False,False
2,0.287,497.0,395.0,False,False,True,False,False


In [175]:
# View Results
view_rand_df(df_dict = mPFC_df_dict, info_df = mPFC_session_info_df)

Dataframe: C58_R20_ChR2_mPFC_30Hz_S02_P04_V_NON_NON
stim_session_list: NV-


,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,session_zone_(v)
0,0.000,NaN,NaN,False,True,False,False,False
1,0.399,475.0,185.0,False,True,False,False,False
2,0.407,475.0,185.0,False,True,False,False,False


In [176]:
FR_session_info_df.head(3)

,session_id,animal_id_1,animal_id_2,virus,stim_target,stim_freq,session_num,pod_num,stim_zone,sens_treat_1,sens_treat_2,stim_session_list
0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,C1000,C42A,ChR2,xPVT,F00,S01,P05,N,NON,NON,NTD
1,C1000_C42A_ChR2_xPVT_F20_S02_P05_T_NON_NON,C1000,C42A,ChR2,xPVT,F20,S02,P05,T,NON,NON,NTD
2,C1000_C42A_ChR2_xPVT_F20_S03_P05_D_NON_NON,C1000,C42A,ChR2,xPVT,F20,S03,P05,D,NON,NON,NTD


In [177]:
mPFC_session_info_df.head(3)

,session_id,animal_id_1,animal_id_2,virus,stim_target,stim_freq,session_num,pod_num,stim_zone,sens_treat_1,sens_treat_2,stim_session_list
0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,C58,R10,ChR2,mPFC,30Hz,S01,P02,N,NON,NON,NVN
1,C58_R10_ChR2_mPFC_30Hz_S02_P02_V_NON_NON,C58,R10,ChR2,mPFC,30Hz,S02,P02,V,NON,NON,NVN
2,C58_R10_ChR2_mPFC_30Hz_S03_P02_N_NON_NON,C58,R10,ChR2,mPFC,30Hz,S03,P02,N,NON,NON,NVN


<div style="background-color:green; color:white; padding:10px; font-size:20px">
4 - Save to S3

In [178]:
from io import StringIO

#### FR

In [179]:
# FR Session Df Dict

s3_path = 's3://behavior-project/processed-data/processed-data/session-df-dictionaries/fr-icss/'

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = s3_path.replace("s3://", "").split('/', 1)

for key, df in FR_df_dict.items():
    # Create a CSV buffer
    csv_buffer = StringIO()

    # Convert the dataframe to CSV and write it to the buffer
    df.to_csv(csv_buffer, index = False)

    # Create a file name based on the dictionary key
    file_name = f"{prefix}/{key}.csv"

    # Upload the CSV file to S3
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())

In [180]:
# Specify the name of your S3 bucket and the key (file path within the bucket)
bucket_name = 'behavior-project'
key = 'processed-data/processed-data/session-info-dfs/FR_session_info_df.csv'

# Create a CSV buffer
csv_buffer = StringIO()

# Convert the dataframe to CSV and write it to the buffer
FR_session_info_df.to_csv(csv_buffer, index = False)

# Upload the CSV file to S3
s3.put_object(Bucket=bucket_name, Key=key, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'EWJR5GF06R6E7Q9Q',
  'HostId': 've6Je70ZvrsZ2r3BjbWn4QbWGiKny+3MULFaQxrLWtgve8OHx5yGTGCjM1tJXDClCvIa9mzk+gs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 've6Je70ZvrsZ2r3BjbWn4QbWGiKny+3MULFaQxrLWtgve8OHx5yGTGCjM1tJXDClCvIa9mzk+gs=',
   'x-amz-request-id': 'EWJR5GF06R6E7Q9Q',
   'date': 'Fri, 28 Jul 2023 18:45:24 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5e7191b001e8b5c559d94cd32104bfe6"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5e7191b001e8b5c559d94cd32104bfe6"',
 'ServerSideEncryption': 'AES256'}

#### mPFC

In [181]:
# mPFC Session Df Dict

s3_path = 's3://behavior-project/processed-data/processed-data/session-df-dictionaries/mpfc-icss/'

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = s3_path.replace("s3://", "").split('/', 1)

for key, df in mPFC_df_dict.items():
    # Create a CSV buffer
    csv_buffer = StringIO()

    # Convert the dataframe to CSV and write it to the buffer
    df.to_csv(csv_buffer, index = False)

    # Create a file name based on the dictionary key
    file_name = f"{prefix}/{key}.csv"

    # Upload the CSV file to S3
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())

In [182]:
# Specify the name of your S3 bucket and the key (file path within the bucket)
bucket_name = 'behavior-project'
key = 'processed-data/processed-data/session-info-dfs/mPFC_session_info_df.csv'

# Create a CSV buffer
csv_buffer = StringIO()

# Convert the dataframe to CSV and write it to the buffer
mPFC_session_info_df.to_csv(csv_buffer, index = False)

# Upload the CSV file to S3
s3.put_object(Bucket=bucket_name, Key=key, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '46HNV8RSQEXDHPT0',
  'HostId': 'qM9hU3XwWwg0jHvJWyLKYeV3Ht7Cf5pL3YrzoTYZUkqJMdQ589WoCYLUHR//GpBqJo/Ufw2f0ww=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qM9hU3XwWwg0jHvJWyLKYeV3Ht7Cf5pL3YrzoTYZUkqJMdQ589WoCYLUHR//GpBqJo/Ufw2f0ww=',
   'x-amz-request-id': '46HNV8RSQEXDHPT0',
   'date': 'Fri, 28 Jul 2023 18:45:29 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"eca0cef1713e43155295145443207e4b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"eca0cef1713e43155295145443207e4b"',
 'ServerSideEncryption': 'AES256'}